In [15]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import re
import json
import validators
from tqdm.notebook import trange, tqdm
from time import sleep
from functools import partial

Se busca obtener noticias de las siguientes categorías:


- catástrofes y accidentes = 4
- cultura y arte = 5
- ecología y planeta = 7

### 1. Crawling

In [8]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]
    
    def __init__(self, query_url):
        self.query_extract_all_href = query_url
        self.all_json_to_send = []
        
    def new_session_get(self):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        return partial(session.get, headers=header)
    
    def start_request(self,url):
        session_get = self.new_session_get()
        response = session_get(url)
        return response
    
    def parse(self, response):
        self.all_json_to_send = []
        all_extracted_href = response.html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send= dict()
            json_to_send["url"]=href
            self.all_json_to_send.append(json_to_send)

In [9]:
def get_urls(seed_crawl, query_url, limit, start=0, leave=False, extra_validation=None):
    crawler = Crawler(query_url)
    url_noticias = []
    for page in trange(start, limit, desc='url noticias', leave=leave):
        response = crawler.start_request(seed_crawl + str(page))
        crawler.parse(response)
        if extra_validation is not None:
            noticias = [obj['url']
                        for obj in crawler.all_json_to_send if extra_validation in obj['url']]
        else:
            noticias = [obj['url'] for obj in crawler.all_json_to_send]
        url_noticias += noticias

    print('total url noticias recopiladas: ', len(url_noticias))
    return url_noticias

def scrap_cnn_news(urls, starting_id, category, sleep_time=0):
    crawler = Crawler('')
    noticias_dicts = []
    for i, url in enumerate(urls):
    # algunas noticias llevan a otros sitios (en planeta pasa)
        if re.match(r'^.*//www.(\w+)\..*', url).groups()[0] == 'cnnchile':
            response = crawler.start_request(url)
            _dict = {'id': i + starting_id,
                     'url': url}
            try:
                _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
                _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
                _dict['category'] = category
                lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
                new_lines = [re.sub(u'\xa0', u' ', line)
                             for line in lines if not line.startswith('Lee también:')]
                new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
                _dict['text'] += ' '.join(new_lines)
                noticias_dicts.append(_dict)
            except IndexError:
                print(i, url)
            sleep(sleep_time)
    return noticias_dicts

In [10]:
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"

In [11]:
SEED_CRAWL = 'https://www.cnnchile.com/category/cultura/page/'
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
STARTING_ID_CULTURA = 18001000

In [12]:
START = 71 # NOTICIAS DE AGOSTO 2020 HACIA ATRAS
LIMIT = 118 # ULTIMA NOTICIA CON FECHA 31-12-2019 18:05
url_noticias_cultura = get_urls(SEED_CRAWL, QUERY_URL,
                                limit=LIMIT, start=71,
                                extra_validation='cultura')

url noticias:   0%|          | 0/47 [00:00<?, ?it/s]

total url noticias recopiladas:  545


In [22]:
import pandas as pd
df_cultura = pd.DataFrame(scrap_cnn_news(url_noticias_cultura,18001000,'cultura'))

In [23]:
df_cultura.head(5)

,id,url,title,text,category
0,18001000,https://www.cnnchile.com/cultura/fechas-reprog...,Ya hay fechas confirmadas: La décima edición d...,"Debido a la actual crisis sanitaria, la produc...",cultura
1,18001001,https://www.cnnchile.com/cultura/el-mix-lanzam...,El Mix: The Rolling Stones y Ozuna lideran los...,La música de Chile y el mundo se reactiva con ...,cultura
2,18001002,https://www.cnnchile.com/cultura/bronko-yotte-...,"Bronko Yotte estrena “Piola”, su nuevo single ...","El tema forma parte de ""Fuero Interno"", el pró...",cultura
3,18001003,https://www.cnnchile.com/cultura/tan-cerca-el-...,"“Tan cerca”, el primer sencillo original de Su...",Constanza Lewin (1° generación) y Constanza Lü...,cultura
4,18001004,https://www.cnnchile.com/cultura/britney-spear...,“El mundo está mirando”: Britney Spears se ref...,"A través de su abogado defensor, la cantante s...",cultura
